# Set Up Environment

## Dependencies

In [2]:
import numpy as np
import pandas as pd
import os
import IPython
import netCDF4 as nc
import  xarray as xr
from datetime import datetime, timedelta

## Set up global variables

In [3]:
os.chdir("..")
# Recognize the current operating system
if os.name == "nt":  # Windows
    delimiter = "\\"
else:  # Unix or Linux
    delimiter = "/"

os.getcwd()

'c:\\Users\\liang.yang\\Desktop\\esmeraldas'

# ML Model

## Organize the dataset and split the trainning set and test set

In [16]:
df_merged = pd.read_csv('data/processed/merged_2023-02-01_2024-03-05.csv', index_col='Unnamed: 0')
# note to mark the first column as the datetime index
df_merged.head()

,ccs_60,ccs_111,ccs_112,ccs_113,ccs_114,ccs_162,ccs_163,ccs_164,ccs_165,ccs_166,...,pixel4450,pixel4451,pixel4452,pixel4453,pixel4454,pixel4506,pixel4507,hidro_level_m1,precip_acumu_sm,hidro_level_sm
2023-02-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.115053,0.446430,0.134628,0.044849,0.001772,0.013029,0.001152,2.32,0.00,5.60
2023-02-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.971292,0.719923,0.030907,0.080944,0.190954,0.012545,0.002356,1.41,0.00,5.64
2023-02-01 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.024778,0.076063,0.000000,0.005566,0.032881,0.000183,0.012840,1.06,0.00,6.43
2023-02-01 09:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000125,0.000008,0.000067,0.000032,0.001298,1.87,0.00,7.10
2023-02-01 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000633,0.001529,1.954425,0.002977,0.001858,0.527266,0.004241,2.39,0.65,7.58


## Add new features

In [17]:
from src.model.features import create_lagged_features
# add a column of Y's roling average
df_merged['y_rolling_avg'] = df_merged['hidro_level_sm'].rolling(window=3, min_periods=1).mean()

# Choose a lag value
max_lag = 5  # For example, you can use a lag of 5 time steps

# Create lagged features for the 'Discharge' column
df_merged = create_lagged_features(df_merged, 'hidro_level_sm', max_lag)

In [18]:
df_y = df_merged['hidro_level_sm']

### Visualize the importance of the features

And filter out the noise pixels

In [8]:
from src.model.train import prepare_data, train_and_evaluate_rf, grid_search
from src.visulization.plot import plot_fitting_result

In [21]:
df_merged = pd.merge(df_y, df_merged_top_100, left_index=True, right_index=True, how='outer') 

In [22]:
# organiza and split the dataset
X_train, X_test, y_train, y_test, dates_train, dates_test = prepare_data(df_merged, train_size=0.7, column_y='hidro_level_sm')

# train and evaluate the model
nse_train, nse_test, mse_train, mse_test, kge_train, kge_test, model, y_train_pred, y_test_pred = train_and_evaluate_rf(X_train, y_train, X_test, y_test, n_estimators=250, 
                                                                                      max_depth=20, min_samples_split=5, min_samples_leaf=5, 
                                                                                      max_features='sqrt', bootstrap=True, n_jobs=-2)
# model, y_train_pred, y_test_pred = grid_search(X_train, y_train, X_test, y_test)
# plot the results
plot_fitting_result(y_train, y_train_pred, dates_train, y_test, y_test_pred, dates_test, 
                     nse_train, mse_train, kge_train, nse_test, mse_test, kge_test, X_train)

Shape of the final dataset is (2722, 101)

Training NSE: 0.980558379997535, MSE: 0.02932594837867079, KGE: 0.9544025417818472
Testing NSE: 0.863924953839946, MSE: 0.09561965941838438, KGE: 0.8067832327822946


Number of training samples: 1905


In [10]:
from src.visulization.plot import plot_feature_importance

In [19]:
# Get the top 100 feature names
feature_importance = plot_feature_importance(X_train, model)
top_100_features = feature_importance['feature'].head(100).tolist()

# Select the columns from df_merged based on the top 100 feature names
df_merged_top_100 = df_merged[top_100_features]

# Training Finihsed, Save the Trained Model Locally

In [ ]:
import joblib
from datetime import datetime


now = datetime.now().strftime("%Y%m%d_%H%M")
joblib.dump(model, f'model_{now}.pkl')



In [23]:
type(model)

sklearn.ensemble._forest.RandomForestRegressor